### Análise de Prestação de Contas Partidárias de Eleições Estaduais

In [31]:
# Importação das libs
import copy
import locale
from locale import atof
import electionsBR as api
import pandas as pd

In [32]:
# Ajuste de formatação
locale.setlocale(locale.LC_ALL, 'pt_BR.UTF-8')
pd.set_option("float_format", locale.currency)
#pd.set_option('display.float_format', lambda x: '%,.2f' % x)

In [48]:
# Leitura dos dados dos bens declarados para um determinado cargo (presidente, governador, senador, deputado federal, deputado estadual, prefeito, vereador)
bens = api.get_assets(year = 2018, position="governador", state = "SP")

In [49]:
# Informações sobre os dados retornados (Linhas, Colunas)
bens.shape

(13403, 22)

In [35]:
# Colunas disponiveis para analise
bens.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13403 entries, 0 to 13402
Data columns (total 22 columns):
 #   Column                          Non-Null Count  Dtype 
---  ------                          --------------  ----- 
 0   DATA_GERACAO                    13403 non-null  object
 1   HORA_GERACAO                    13403 non-null  object
 2   ANO_ELEICAO                     13403 non-null  object
 3   DESCRICAO_ELEICAO               13403 non-null  object
 4   SIGLA_UF                        13403 non-null  object
 5   SEQUENCIAL_CANDIDATO            13403 non-null  object
 6   CD_TIPO_BEM_CANDIDATO           13403 non-null  object
 7   DS_TIPO_BEM_CANDIDATO           13403 non-null  object
 8   DETALHE_BEM                     13403 non-null  object
 9   VALOR_BEM                       13403 non-null  object
 10  DATA_ULTIMA_ATUALIZACAO         13403 non-null  object
 11  HORA_ULTIMA_ATUALIZACAO         13403 non-null  object
 12  ID_CANDIDATO                    13403 non-null

In [42]:
# Exibição de um pequena amostra
bens.sample(5)

,DATA_GERACAO,HORA_GERACAO,ANO_ELEICAO,DESCRICAO_ELEICAO,SIGLA_UF,SEQUENCIAL_CANDIDATO,CD_TIPO_BEM_CANDIDATO,DS_TIPO_BEM_CANDIDATO,DETALHE_BEM,VALOR_BEM,...,ID_CANDIDATO,SIGLA_UE,CODIGO_CARGO,DESCRICAO_CARGO,NUMERO_PARTIDO,SIGLA_PARTIDO,NOME_CANDIDATO,NUMERO_CANDIDATO,CPF_CANDIDATO,NUM_TITULO_ELEITORAL_CANDIDATO
1663,13/04/2019,19:06:57,2018,Eleições Gerais Estaduais 2018,SP,250000602874,11,Apartamento,#NULO#,"166.053,22",...,10012720,SP,6,DEPUTADO FEDERAL,51,PATRI,LUIZ CARLOS MOTA,5103,88676501815,045909010167
4984,13/04/2019,19:06:57,2018,Eleições Gerais Estaduais 2018,SP,250000605761,41,Caderneta de poupança,#NULO#,"18.455,46",...,10007417,SP,7,DEPUTADO ESTADUAL,55,PSD,KARINA CARLA DA SILVA,55900,17407113866,162232190159
11214,13/04/2019,19:06:57,2018,Eleições Gerais Estaduais 2018,SP,250000619500,15,Sala ou conjunto,#NULO#,"18.500,00",...,10016173,SP,7,DEPUTADO ESTADUAL,22,PR,SERGIO ANTONIO DONHA,22133,03041714841,037950960175
11430,13/04/2019,19:06:57,2018,Eleições Gerais Estaduais 2018,SP,250000623474,21,"Veículo automotor terrestre: caminhão, automóv...",AUTOMOVEL,"50.000,00",...,10006585,SP,6,DEPUTADO FEDERAL,12,PDT,JOSE CARLOS OROSCO,1212,27098424838,253970790124
12127,13/04/2019,19:06:57,2018,Eleições Gerais Estaduais 2018,SP,250000623813,21,"Veículo automotor terrestre: caminhão, automóv...",#NULO#,"178.000,00",...,10013736,SP,7,DEPUTADO ESTADUAL,19,PODE,PEDRO MASSAMI KIKUDOME,19123,00647487829,080932040159


In [50]:
# Soma do valor total de bens declarados por cada candidato
bens_number = copy.deepcopy(bens)
bens_number["VALOR_BEM"] = bens_number["VALOR_BEM"].map(atof)
bens_number["VALOR_BEM"] = pd.to_numeric(bens_number["VALOR_BEM"])
bens_number.groupby("CPF_CANDIDATO")["VALOR_BEM"].sum().sort_values(ascending=False)

CPF_CANDIDATO
01654727822   R$ 1202006700,00
08045481889   R$ 1018384000,00
17250615890    R$ 600000000,00
25569763813    R$ 340000000,00
94062897849    R$ 189859904,76
                    ...       
43784174809            R$ 8,25
01941114822            R$ 5,91
19346311835            R$ 1,00
94126259872            R$ 1,00
06361791840            R$ 0,20
Name: VALOR_BEM, Length: 2421, dtype: float64